This file contains some functions to check that ICD OPERTN, MAINSPEF and PROCODES codes match with the reference tables. Functions are tested on APC tables (positive and negative). I assume the tables from different years are merged.

# Connection

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import numpy as np

%matplotlib inline

In [2]:
def connect():
    return pyodbc.connect(
        'DRIVER={/usr/lib/x86_64-linux-gnu/odbc/libtdsodbc.so}; SERVER=192.168.5.78;'
        'DATABASE=IMS-NHSDigital;UID=sa;PWD=2HsxpmkDLSlHlT;port=1433;'
        'TDS_Version=8.0;')
engine = sqlalchemy.create_engine('mssql+pyodbc://', creator=connect)
conn = engine.connect()

/home/vsalvatelli/anaconda3/lib/python3.6/site-packages/sqlalchemy/connectors/pyodbc.py:92: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  "No driver name specified; "


# Check on PROCODES

Extract the unique set of hospitals (PROCODE) codes and provide patient count for each. Join this with the reference table for the PROCODE codes and flag those that do not match a code in the reference table.

In [104]:
#extract codes
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

label_ID="PROCODE3"

unique_procodes=set()
for tablename in table_list:
    query= 'SELECT ' + label_ID  +' from ' + tablename
    print(query)
    procodes=pd.read_sql(query, conn)[label_ID]
    unique_procodes.update(procodes)
    print(len(unique_procodes))

#print(unique_procodes)
#I suggest to count for those not matching if we just care about them

SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201299
135
SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201399
158
SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201499
172
SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201599
181
SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201613
188


In [88]:
#if I use the procode col the majority of codes are 5 digits,
#sometime endings with 00/01, sometimes with other letters or number
list(unique_procodes)[0:12]

['RXE',
 'RTG',
 'NT2',
 'RTX',
 'RGR',
 'RXQ',
 'RHM',
 'RR7',
 'RKB',
 'REN',
 'RAN',
 'RM3']

In [52]:
#reading reference codes
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimProvider.csv'
ref_label_procode='ProviderCode'
ref_procode=pd.read_csv(ref_table, delimiter="|")[ref_label_procode].unique()

/home/vsalvatelli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
#the majority of reference codes are 3digit codes, seems sensible to use the procode3 from HES tables
ref_procode[:30]

array(['------------', 'Missing', 'Unknown', '5A3', '5A4', '5A5', '5AT',
       '5C1', '5C2', '5C3', '5C4', '5C5', '5C9', '5CN', '5CQ', '5F1',
       '5FE', '5FL', '5GC', '5H8', '5J2', '5J9', '5JE', '5K3', '5K5',
       '5K7', '5KG', '5KL', '5L1', '5L3'], dtype=object)

In [43]:
len(set(ref_procode))

452

In [66]:
#some codes are not matching
unique_procodes.issubset(set(ref_procode))

False

In [89]:
#about 1% of coding is not matching
#if I use procode is about 90% ...
NM_procodes=[x for x in unique_procodes if x not in ref_procode]
print(len(NM_procodes)/len(unique_procodes))
print(NM_procodes)

0.010638297872340425
['8E3', '8A1']


In [84]:
ref_code_8=[x for x in ref_procode if x.startswith('8')]

In [85]:
ref_code_8

['8A101',
 '8E339',
 '8F689',
 '8HP32',
 '8HP46',
 '8HT55',
 '8HT56',
 '8J124',
 '8J244',
 '8J248',
 '8J285',
 '8G301']

In [86]:
#we might assume that 8E3 stands for 8A101 and 8E3 stands for 8E339
#for what we use these codes after? useful to join with Provider Caption (contains the name of the provider)?

In [151]:
#count occurencies for not matching codes
NM_df=pd.DataFrame({label_ID: [],'counts': []})
for code in NM_procodes:
    for tablename in table_list:
        query= 'SELECT ' + label_ID + ', COUNT(' + label_ID + ') AS counts' + ' from ' + tablename + \
        ' WHERE '+ label_ID + ' IS NOT NULL AND ' + label_ID +' = \'' + code + '\' GROUP BY ' + label_ID + ';'  
        #print(query)
        NM_df=NM_df.append(pd.read_sql(query, conn), ignore_index=True)

In [156]:
#grouping the counts from different tables
NM_df.groupby(label_ID)['counts'].sum()

PROCODE3
8A1    10.0
8E3     3.0
Name: counts, dtype: float64